## Import external modules 

In [1]:
from __future__ import division
#data_path = ""

## Counting assets

Let's create a pyhton dictionary for storing the number of times **each asset** appears in the customers portfolios and print it.

In [2]:
asset_counts = {}

with open("data/Assets_Counter_Anonimized.txt", 'r') as f:

    for line in f:
        items = line.split('\t')
        asset, count = items[0], float(items[1].strip())
        asset_counts[asset] = count

#print(asset_counts)

## Counting pairs

Let's create a python dictionary for storing the number of times **each asset pair** appears in the customers portfolios and print it.

In [3]:
dict_pairs_counts = {}

f_to_read_path = 'data/Assets_Pairs_Anonimized.txt'
pairs_file = open(f_to_read_path,'r')
pair_row = pairs_file.readline()

while (pair_row != ""):

    asset_fields = pair_row.split('\t')

    a_i = asset_fields[0].strip()
    a_j = asset_fields[1].strip()
    a_ij= a_i + "," + a_j
    
    if (a_ij not in dict_pairs_counts):
        dict_pairs_counts[a_ij] = 1
    else:
        dict_pairs_counts[a_ij] += 1
        
    pair_row = pairs_file.readline()

pairs_file.close()
#print dict_pairs_counts

## Computing similarities

Let's create a dictionary containing the **similarities for each pair of asset observed in the customers portolios**.

In [4]:
pairs_similarity = {}

for a_ij in dict_pairs_counts:

    asset_pair = str(a_ij).split(',')
    a_i = asset_pair[0].strip()
    a_j = asset_pair[1].strip()

    sim_ij = 2 * dict_pairs_counts[a_ij] / (asset_counts[a_i] + asset_counts[a_j])

    pair_key1 = a_i + "," + a_j
    pairs_similarity[pair_key1] = sim_ij
    
    pair_key2 = a_j + "," + a_i
    pairs_similarity[pair_key2] = sim_ij


In [5]:
#pairs_similarity

## Building a dict() of dict()

Now we are going to build a dictionay of dictionries. The key of the primary dictionay will be a given asset and the value of the primary dictionay will be a secondary dictionay. Each key of the secondary dictionay is an asset and the corresponding value is the similarity between the primay dictionary key and the secondary dictionay key.

In [6]:
asset_dict = {}

for pair_ij in pairs_similarity:
    
    assets = str(pair_ij).split(',')
    a_i = assets[0].strip()
    a_j = assets[1].strip()

    if a_i not in asset_dict:
        # assign an empty dictionay if the i-th asset's not present
        asset_dict[a_i] = {}
        # assign the value of pair_ij to the key a_j of the subdict() of a_i
        asset_dict[a_i ][a_j] = pairs_similarity[pair_ij]
    else: 
        # assign the value of pair_ij to the key a_i of the subdict() of a_j
        asset_dict[a_i][a_j] = pairs_similarity[pair_ij]
        
    if a_j not in asset_dict:
        # assign an empty dictionay id the j-th asset is not observed yet
        asset_dict[a_j] = {}
        # assign the value of pair_ij to the key a_i of the subdict() of a_j
        asset_dict[a_j][a_i] = pairs_similarity[pair_ij]
    
    else:
        # assign the value of pair_ij to the key a_j of the subdict() of a_i
        asset_dict[a_j][a_i] = pairs_similarity[pair_ij]



In [7]:
#asset_dict

## Ordering the dict() of dict() and saving to file

In [8]:
import operator, json

# sort the dictionary of similarities
for pairs in asset_dict:
    asset_dict[pairs] = sorted(asset_dict[pairs].items(),
                               key = operator.itemgetter(1),
                               reverse = True)
    
# save the dictionary into a json file
json.dump(asset_dict,open('output_files/sorted_asset_dict.txt', "w"))

## Using the dict() of dict()

In [9]:
d = json.load(open('output_files/sorted_asset_dict.txt'))

def suggest_me(asset):
    if asset in d:
        counter = 0
        for i in d[asset]:
            if counter<10:
                print str(i[0]),", rank: ",i[1]
                counter += 1
    else:
        print "Asset not found"

In [10]:
suggest_me('1')

7 , rank:  0.4
247 , rank:  0.117647058824
8 , rank:  0.0952380952381
302 , rank:  0.0740740740741
334 , rank:  0.0434782608696
32 , rank:  0.0425531914894
364 , rank:  0.0294117647059
349 , rank:  0.0204081632653
10 , rank:  0.0194174757282
412 , rank:  0.013698630137


In [11]:
d['1']

[[u'7', 0.4],
 [u'247', 0.11764705882352941],
 [u'8', 0.09523809523809523],
 [u'302', 0.07407407407407407],
 [u'334', 0.043478260869565216],
 [u'32', 0.0425531914893617],
 [u'364', 0.029411764705882353],
 [u'349', 0.02040816326530612],
 [u'10', 0.019417475728155338],
 [u'412', 0.0136986301369863],
 [u'109', 0.011111111111111112],
 [u'279', 0.010471204188481676],
 [u'655', 0.00904977375565611],
 [u'47', 0.008032128514056224],
 [u'43', 0.006134969325153374],
 [u'689', 0.005633802816901409],
 [u'177', 0.003129890453834116],
 [u'772', 0.00045599635202918376],
 [u'765', 0.00036771465342893915],
 [u'75', 0.00014141271300289896]]